# I. Network Optimization
## [ Optimal Tour Route from Spain to Finland ]

## Introduction

In this part, we aimed to design an optimal tour route from Spain to Finland (The most distanced country from Spain) that connects various European capital cities. The challenge was to minimize the total distance traveled while considering the actual distances and connectivity between the cities.

We defined the nodes (cities) and edges (routes between cities) as sets in our model. The decision variables included binary variables representing whether a route between two cities was included in the tour.
The Minimum Cost Flow algorithm was applied to this network of cities (nodes) and routes (edges), where the cost was associated with the distance between cities. 

We implemented flow Conservation to ensure that each city was visited exactly once and subtour Elimination using Miller-Tucker-Zemlin (MTZ) constraints.

In [ ]:
import  pandas as pd
from pyomo.environ import ConcreteModel, Set, Var, Objective, Constraint, SolverFactory, Binary, NonNegativeReals, minimize



file_path = 'A:/AI_STUDY/Practice/2023-2024 MADRID/Optimization and analytics/HW2/distances_matrix.csv'
distances_df = pd.read_csv(file_path)

distances_df.head()



,Unnamed: 0,Berlin,Paris,Rome,Madrid,Warsaw,Bucharest,Amsterdam,Brussels,Athens,...,Bratislava,Dublin,Zagreb,Vilnius,Ljubljana,Riga,Tallinn,Nicosia,Luxembourg,Valletta
0,Berlin,0.0,876.0,1181.0,1866.0,518.0,1295.0,573.0,648.0,1804.0,...,552.0,1314.0,769.0,819.0,723.0,845.0,1042.0,2490.0,600.0,1850.0
1,Paris,876.0,0.0,1105.0,1051.0,1367.0,1870.0,430.0,263.0,2097.0,...,1088.0,779.0,1079.0,1695.0,964.0,1703.0,1858.0,2950.0,287.0,1749.0
2,Rome,1181.0,1105.0,0.0,1363.0,1315.0,1137.0,1295.0,1172.0,1052.0,...,783.0,1884.0,515.0,1701.0,488.0,1865.0,2124.0,1956.0,988.0,690.0
3,Madrid,1866.0,1051.0,1363.0,0.0,2289.0,2471.0,1481.0,1315.0,2369.0,...,1857.0,1450.0,1699.0,2659.0,1596.0,2712.0,2892.0,3283.0,1278.0,1665.0
4,Warsaw,518.0,1367.0,1315.0,2289.0,0.0,945.0,1092.0,1159.0,1599.0,...,532.0,1825.0,803.0,392.0,834.0,560.0,833.0,2134.0,1080.0,1887.0


In [ ]:
# Extracting the nodes (cities) from the CSV file
nodes = distances_df.columns[1:].tolist()

# Defining the edges based on the given connections between countries
#       We'll assume the capital city of each country as the node.
country_connections = [
    ("Spain", "Italy"), ("Spain", "France"), ("Spain", "Portugal"),
    ("Portugal", "Ireland"), ("France", "Ireland"), ("France", "Italy"),
    ("France", "Luxembourg"), ("France", "Belgium"), ("France", "Germany"),
    ("Italy", "Austria"), ("Italy", "Slovenia"), ("Italy", "Croatia"),
    ("Italy", "Greece"), ("Italy", "Malta"), ("Greece", "Bulgaria"),
    ("Greece", "Cyprus"), ("Bulgaria", "Romania"), ("Romania", "Hungary"),
    ("Romania", "Poland"), ("Croatia", "Slovenia"), ("Croatia", "Hungary"),
    ("Slovenia", "Austria"), ("Slovenia", "Hungary"), ("Austria", "Hungary"),
    ("Austria", "Slovakia"), ("Austria", "Czech Republic"), ("Austria", "Germany"),
    ("Luxembourg", "Germany"), ("Belgium", "Luxembourg"), ("Belgium", "Netherlands"),
    ("Belgium", "Germany"), ("Netherlands", "Germany"), ("Netherlands", "Denmark"),
    ("Germany", "Czech Republic"), ("Germany", "Denmark"), ("Germany", "Poland"),
    ("Czech Republic", "Poland"), ("Slovakia", "Czech Republic"), ("Slovakia", "Poland"),
    ("Denmark", "Sweden"), ("Poland", "Sweden"), ("Poland", "Lithuania"),
    ("Lithuania", "Latvia"), ("Sweden", "Finland"),
    ("Estonia", "Finland"), ("Estonia", "Latvia")
]

# Capital cities for each country (as used in the CSV)
capital_cities = {
    "Spain": "Madrid", "Italy": "Rome", "France": "Paris", "Portugal": "Lisbon",
    "Ireland": "Dublin", "Luxembourg": "Luxembourg", "Belgium": "Brussels",
    "Germany": "Berlin", "Austria": "Vienna", "Slovenia": "Ljubljana",
    "Croatia": "Zagreb", "Greece": "Athens", "Malta": "Valletta", "Bulgaria": "Sofia",
    "Cyprus": "Nicosia", "Romania": "Bucharest", "Hungary": "Budapest",
    "Poland": "Warsaw", "Netherlands": "Amsterdam", "Denmark": "Copenhagen",
    "Czech Republic": "Prague", "Slovakia": "Bratislava", "Sweden": "Stockholm",
    "Lithuania": "Vilnius", "Latvia": "Riga", "Finland": "Helsinki", "Estonia" : "Tallinn"
}

# Creating edges and their costs
edges = []
for connection in country_connections:
    city1 = capital_cities[connection[0]]
    city2 = capital_cities[connection[1]]
    distance = distances_df.loc[distances_df['Unnamed: 0'] == city1, city2].values[0]
    # Adding edge from city1 to city2 and vice versa (bidirectional)
    edges.append((city1, city2, distance))
    edges.append((city2, city1, distance))


edges[:10]


[('Madrid', 'Rome', 1363.0),
 ('Rome', 'Madrid', 1363.0),
 ('Madrid', 'Paris', 1051.0),
 ('Paris', 'Madrid', 1051.0),
 ('Madrid', 'Lisbon', 504.0),
 ('Lisbon', 'Madrid', 504.0),
 ('Lisbon', 'Dublin', 1640.0),
 ('Dublin', 'Lisbon', 1640.0),
 ('Paris', 'Dublin', 779.0),
 ('Dublin', 'Paris', 779.0)]

## Model Formulation

#### **Decision Variables**:
- $x_{ij}$: Binary variable that is 1 if the flow is from node $i$ to node $j$, and 0 otherwise.
- $u_i$: Continuous variable used for subtour elimination, representing the position of node $i$ in the path.

#### **Objective Function**:
Minimize the total distance of the selected path:
$$
\text{Minimize } Z = \sum_{(i, j) \in E} c_{ij} x_{ij}
$$
where $c_{ij}$ is the cost of the edge from node $i$ to node $j$ and $E$ is the set of all edges.

#### **Constraints**:

#### 1. Flow Conservation Constraints:
For each node $i$ (except Madrid and Helsinki), ensure the inflow equals the outflow:
$$
\sum_{\{j \mid (j,i) \in E\}} x_{ji} - \sum_{\{j \mid (i,j) \in E\}} x_{ij} =
\begin{cases}
1 & \text{if } i = \text{ Madrid} \\
-1 & \text{if } i = \text{ Helsinki} \\
0 & \text{otherwise}
\end{cases}
$$

#### 2. Subtour Elimination Constraints (MTZ Constraints):
For each pair of nodes $(i, j)$ (excluding the source Madrid and sink Helsinki):
$$
u_i - u_j + |N| \cdot x_{ij} \leq |N| - 1 \quad \forall (i, j) \in E, i \neq \text{Madrid}, j \neq \text{Helsinki}
$$
where $|N|$ is the number of nodes in the network, and $u_i$ represents the order of node $i$ in the path.


In [ ]:
# Instantiate the model
model = ConcreteModel()

# Define sets based on nodes and edges
model.nodes = Set(initialize=nodes)
model.edges = Set(dimen=2, initialize=[(e[0], e[1]) for e in edges])

# Define the decision variables for flow and the additional variables for subtour elimination (MTZ constraints)
model.x = Var(model.edges, within=Binary)
model.u = Var(model.nodes, within=NonNegativeReals, bounds=(0, len(model.nodes)-1))

# Define the costs for each edge
model.Costs = {e[:2]: e[2] for e in edges}

# Define the objective function to minimize total distance
def obj_rule(m):
    return sum(m.Costs[e] * m.x[e] for e in m.edges)
model.objective = Objective(rule=obj_rule, sense=minimize)

# Define flow conservation constraints
def flow_rule(m, n):
    return (sum(m.x[i, n] for i in m.nodes if (i, n) in m.edges) -
            sum(m.x[n, j] for j in m.nodes if (n, j) in m.edges) ==
            (1 if n == 'Madrid' else -1 if n == 'Helsinki' else 0))
model.flow = Constraint(model.nodes, rule=flow_rule)

# Define subtour elimination constraints (MTZ constraints)
def mtz_rule(m, i, j):
    if i != 'Madrid' and j != 'Helsinki' and (i, j) in m.edges:
        return m.u[i] - m.u[j] + len(m.nodes) * m.x[i, j] <= len(m.nodes) - 1
    else:
        return Constraint.Skip
model.mtz = Constraint(model.nodes, model.nodes, rule=mtz_rule)


In [ ]:
!glpsol --version

GLPSOL--GLPK LP/MIP Solver 5.0
Copyright (C) 2000-2020 Free Software Foundation, Inc.

This program has ABSOLUTELY NO WARRANTY.

This program is free software; you may re-distribute it under the terms
of the GNU General Public License version 3 or later.


In [ ]:
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)


# Display the results
model.display()


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\min\AppData\Local\Temp\tmpg39s9qr4.glpk.raw --wglp C:\Users\min\AppData\Local\Temp\tmprs01rxwq.glpk.glp
 --cpxlp C:\Users\min\AppData\Local\Temp\tmp0h2x_5bh.pyomo.lp
Reading problem data from 'C:\Users\min\AppData\Local\Temp\tmp0h2x_5bh.pyomo.lp'...
C:\Users\min\AppData\Local\Temp\tmp0h2x_5bh.pyomo.lp:1008: warning: lower bound of variable 'x2' redefined
C:\Users\min\AppData\Local\Temp\tmp0h2x_5bh.pyomo.lp:1008: warning: upper bound of variable 'x2' redefined
114 rows, 119 columns, 445 non-zeros
92 integer variables, all of which are binary
1100 lines were read
Writing problem data to 'C:\Users\min\AppData\Local\Temp\tmprs01rxwq.glpk.glp'...
889 lines were written
GLPK Integer Optimizer 5.0
114 rows, 119 columns, 445 non-zeros
92 integer variables, all of which are binary
Preprocessing...
114 rows, 119 columns, 445 non-zeros
92 integer variables, all of which are binary
Scaling...
 A: min|aij| 

## Result Interpretation
As a result the solver found an optimal solution with a total tour distance of 3024.0 units which is the minimum possible distance to traverse all the designated capital cities while adhering to our constraints.
Moreover, the results show a selection of routes between cities. For example, the route from 'Amsterdam' to 'Brussels' has a value of 1.0, indicating its inclusion in the optimal tour. However, a value of 0.0, such as in the route from 'Amsterdam' to 'Berlin', denotes its exclusion from the optimal path.

Let's print only the edges that are included in the optimal route.

In [ ]:
# Print the results
for edge in model.edges:
    if model.x[edge].value > 0:
        print(f"Flow on edge {edge}: {model.x[edge].value}")

Flow on edge ('Paris', 'Madrid'): 1.0
Flow on edge ('Brussels', 'Paris'): 1.0
Flow on edge ('Amsterdam', 'Brussels'): 1.0
Flow on edge ('Copenhagen', 'Amsterdam'): 1.0
Flow on edge ('Stockholm', 'Copenhagen'): 1.0
Flow on edge ('Helsinki', 'Stockholm'): 1.0


Based on the results, we can conclude that the optimal route to travel from Spain to Finland is **'Madrid - Paris - Brussels - Amsterdam - Copenhagen - Stockholm - Helsinki'**. 

Further in the future, there is potential to enrich this model by incorporating transportation data. This might include analyzing the efficiency of different modes of transportation, such as vehicles, trains, and flights, particularly in terms of fuel consumption. While we were unable to perform this analysis due to limited time and resources, we believe that integrating such data could lead to more practical and applicable optimization solutions in the tourism industry.



# II. Non Linear Optimization
##  [ Optimization of Pricing Strategies for Christmas Trees ]

# 🎄

## Introduction
There are items that the demands of it are elastic and inelastic.
Among Christmas trees are mostly in every household so it its demand is inelastic, however, the creteria to choose among the size of the tree, naturality of the tress (artifical tree or natural tree) can be differ. So we looked up on the price - demand data, production, cost data and made up the equations that might realistically reflect the real world problem.


In retail economics, the concept of demand elasticity is pivotal, particularly for seasonal items like Christmas trees. While the general demand for these trees is quite inelastic due to their cultural significance, variations in consumer preferences based on size, type (artificial or natural), and price can significantly influence specific demand patterns. This scenario presents a unique opportunity to search into demand curve formulation and optimization techniques.

In our model, we use a demand curve formulation to represent the relationship between the price of each type of Christmas tree and the corresponding consumer demand. The demand curves for the three types of trees are modeled using custom functions that account for price sensitivity and cross-price effects.


We define variables (x1, x2, x3) representing the prices of three different types of Christmas trees.
Demand functions (D, D2, D3) are formulated based on these prices, incorporating aspects of price elasticity and interdependence of demand among different tree types.
Production (P) and cost (C) functions are established to reflect the economic realities of tree production, emphasizing the inverse relationship between production volume and per-unit cost.


## Modle Formulation

#### **Variables**
- $x_1$: Price for tree type 1 (Normal Sized Artificial Trees)
- $x_2$: Price for tree type 2 (Normal Size Natural Trees)
- $x_3$ : Price for tree type 3 (Large Size Natural Trees)

#### **Demand Functions**
- For tree type 1: $D(x_1, x_2, x_3) = 100 - 2x_1 + 0.25(x_2 + x_3)$
- For tree type 2: $D2(x_2, x_1, x_3) = 300 - x_2^2 + 0.25(x_1 + x_3)$
- For tree type 3: $D3(x_3, x_2, x_1) = 50 - x_3^{0.8} + 0.25(x_2 + x_1)$

#### **Production Function**
- $P(x) = 1.2 \cdot x$

#### **Cost Function**
- $C(x) = 6000 \cdot x^{-2}$

#### **Objective Function**
- Objective: Maximize profit
- Profit Formula:
  $$
  \text{Profit} =
  D(x_1, x_2, x_3) \cdot (x_1 - C(P(D(x_1, x_2, x_3)))) +
  D2(x_2, x_1, x_3) \cdot (x_2 - C(P(D2(x_2, x_1, x_3)))) +
  D3(x_3, x_2, x_1) \cdot (x_3 - C(P(D3(x_3, x_2, x_1)))) - 50
  $$

#### **Constraints**
1. Non-negative Demand:
   - $D(x_1, x_2, x_3) \geq 0$
   - $D2(x_2, x_1, x_3) \geq 0$
   - $D3(x_3, x_2, x_1) \geq 0$
2. Minimum Sale Requirement (Local Government Rule):
   - $D(x_1, x_2, x_3) + D2(x_2, x_1, x_3) + D3(x_3, x_2, x_1) \geq 300$
3. Budget Constraint (For Stocking Up Trees):
   - $D(x_1, x_2, x_3) \cdot C(P(D(x_1, x_2, x_3))) + D2(x_2, x_1, x_3) \cdot C(P(D2(x_2, x_1, x_3))) + D3(x_3, x_2, x_1) \cdot C(P(D3(x_3, x_2, x_1))) \leq 300$



In [2]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import os

os.environ['NEOS_EMAIL'] = 'min7575675@gmail.com'
M = ConcreteModel()

M.x1 = Var(within = NonNegativeReals, initialize=1.0, bounds=(1e-6, None))  # create x1
M.x2 = Var(within = NonNegativeReals, initialize=1.0)  # create x2
M.x3 = Var(within = NonNegativeReals, initialize=1.0)  # create x3

# demand for tree type 1 depending on the price
def D(x,y,z):
    return 300-x**2+0.25*(y+z)
# demand for tree type 2
def D2(x,y,z):
    return 100-2*x+0.25*(y+z)
# demand for tree type 3
def D3(x,y,z):
    return 50-x**0.8+0.25*(y+z)
# production for any tree given demand
def P(x):
    return 1.2*x
# cost for any tree given their production (low production = high per unit cost, high production = low per unit cost)
def C(x):
    return (6000*x**-2)

# define objective
# obj = maximise profit
# sum (Demand for tree * (price - cost for tree))
#
M.obj = Objective(
    expr= D(M.x1,M.x2,M.x3)*(M.x1-C(P(D(M.x1,M.x2,M.x3))))+D2(M.x2,M.x1,M.x3)*(M.x2-C(P(D2(M.x2,M.x1,M.x3))))+D3(M.x3,M.x2,M.x1)*(M.x3-C(P(D3(M.x3,M.x2,M.x1))))-50,
    sense=maximize)

# constraints
# we dont want our demands to be below 0
M.cons = Constraint(expr = D(M.x1,M.x2,M.x3) >= 0)
M.cons2 = Constraint(expr = D2(M.x2,M.x1,M.x3) >= 0)
M.cons3 = Constraint(expr = D3(M.x3,M.x2,M.x1) >= 0)
# there is a rule by the local government that if a shop occupies a spot they have sell atleast 300 trees
M.cons4 = Constraint(expr = D(M.x1,M.x2,M.x3)+D2(M.x2,M.x1,M.x3)+D3(M.x3,M.x2,M.x1) >= 300)
# hte company has an initial budget of 300 units of currency to stock up on trees
M.cons5 = Constraint(expr = D(M.x1,M.x2,M.x3)*C(P(D(M.x1,M.x2,M.x3)))+D2(M.x2,M.x1,M.x3)*C(P(D2(M.x2,M.x1,M.x3)))+D3(M.x3,M.x2,M.x1)*C(P(D3(M.x3,M.x2,M.x1))) <= 300)


#solver.solve(M)
# If you have problems with ipopt locally, try the NEOS Server (from the class)
opt = SolverFactory('ipopt')  # Select solver
solver_manager = SolverManagerFactory('neos')  # Solve in neos server
solver_manager.solve(M, opt=opt)

# obtain the solution
#solver = SolverFactory("ipopt") # define the solver
M.dual = Suffix(direction=Suffix.IMPORT)
#solution = solver.solve(M) # solve

# Display solution
print('x1=',M.x1())
print('x2=',M.x2())
print('x3=',M.x3())
print('constraint=',M.cons5())

# Display lagrange multipliers
display(M.dual)

x1= 10.42806311719589
x2= 37.889370741594924
x3= 100.96381280119208
constraint= 288.1826486276409
dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key : Value


## Result Interpretation

As the result of the model the Optimal Pricings based on each type are like the following. 

**Tree Type 1 (x1)**: The optimal price is approximately €10.43. This price point is likely a balance between maintaining a reasonable demand and ensuring profitability.
**Tree Type 2 (x2)**: The optimal price for this type is significantly higher, at around €37.89. This suggests that Tree Type 2 has a more inelastic demand compared to Tree Type 1, allowing for a higher price without a substantial drop in demand.
**Tree Type 3 (x3)**: At about €100.96, this tree type is priced much higher than the others. 

**The value of the constraint is approximately €288.18**, which is under the budget limit of €300. This indicates that the model successfully optimized the prices while staying within the budgetary constraints for stocking the trees.

The dual values(Lagrange Multipliers) associated with the constraints would provide insights into the sensitivity of the objective function to the constraints. These values help in understanding how much the objective function (profit) would improve if the constraints were relaxed or tightened. Unfortunately, Felix and Min both encountered issues running the IPOPT solver locally. We opted to use the NEOS server for solving the problem. However, a limitation of this approach was that the dual values were not printed in the output, restricting our ability to perform a detailed sensitivity analysis.